## Importing necessary packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# preprocessing/decomposition
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, KernelPCA

# keras 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint

# model evaluation
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# supportive models
from sklearn.ensemble import GradientBoostingRegressor
# feature selection (from supportive model)
from sklearn.feature_selection import SelectFromModel

# to make results reproducible
seed = 42 # was 42

Using Theano backend.
Using cuDNN version 5105 on context None
Mapped name None to device cuda: TITAN X (Pascal) (0000:03:00.0)


In [2]:
# Read datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# save IDs for submission
id_test = test['ID'].copy()

In [4]:
# glue datasets together
total = pd.concat([train, test], axis=0)
print('initial shape: {}'.format(total.shape))

# binary indexes for train/test set split
is_train = ~total.y.isnull()

initial shape: (8418, 378)


In [5]:
# find all categorical features
cf = total.select_dtypes(include=['object']).columns
print total[cf].head()

   X0 X1  X2 X3 X4 X5 X6 X8
0   k  v  at  a  d  u  j  o
1   k  t  av  e  d  y  l  o
2  az  w   n  c  d  x  j  x
3  az  t   n  f  d  x  l  e
4  az  v   n  f  d  h  d  n


In [6]:
# make one-hot-encoding convenient way - pandas.get_dummies(df) function
dummies = pd.get_dummies(
    total[cf],
    drop_first=False # you can set it = True to ommit multicollinearity (crucial for linear models)
)

print('oh-encoded shape: {}'.format(dummies.shape))
print dummies.head()

oh-encoded shape: (8418, 211)
   X0_a  X0_aa  X0_ab  X0_ac  X0_ad  X0_ae  X0_af  X0_ag  X0_ai  X0_aj  ...   \
0     0      0      0      0      0      0      0      0      0      0  ...    
1     0      0      0      0      0      0      0      0      0      0  ...    
2     0      0      0      0      0      0      0      0      0      0  ...    
3     0      0      0      0      0      0      0      0      0      0  ...    
4     0      0      0      0      0      0      0      0      0      0  ...    

   X8_p  X8_q  X8_r  X8_s  X8_t  X8_u  X8_v  X8_w  X8_x  X8_y  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     1     0  
3     0     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     0  

[5 rows x 211 columns]


In [7]:
# get rid of old columns and append them encoded
total = pd.concat(
    [
        total.drop(cf, axis=1), # drop old
        dummies # append them one-hot-encoded
    ],
    axis=1 # column-wise
)

print('appended-encoded shape: {}'.format(total.shape))

appended-encoded shape: (8418, 581)


In [8]:
# recreate train/test again, now with dropped ID column
train, test = total[is_train].drop(['ID'], axis=1), total[~is_train].drop(['ID', 'y'], axis=1)

# drop redundant objects
del total

# check shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))


Train shape: (4209, 580)
Test shape: (4209, 579)


In [9]:
# Calculate additional features: dimensionality reduction components
n_comp=10 # was 10

# uncomment to scale data before applying decompositions
# however, all features are binary (in [0,1] interval), i don't know if it's worth something
train_scaled = train.drop('y', axis=1).copy()
test_scaled = test.copy()
'''
ss = StandardScaler()
ss.fit(train.drop('y', axis=1))

train_scaled = ss.transform(train.drop('y', axis=1).copy())
test_scaled = ss.transform(test.copy())
'''

# PCA
pca = PCA(n_components=n_comp, random_state=seed)
pca2_results_train = pca.fit_transform(train_scaled)
pca2_results_test = pca.transform(test_scaled)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train_scaled)
ica2_results_test = ica.transform(test_scaled)

# Factor Analysis
fca = FactorAnalysis(n_components=n_comp, random_state=seed)
fca2_results_train = fca.fit_transform(train_scaled)
fca2_results_test = fca.transform(test_scaled)

# Append it to dataframes
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
    #train['fca_' + str(i)] = fca2_results_train[:,i-1]
    #test['fca_' + str(i)] = fca2_results_test[:, i-1]
   

/home/krohitm/anaconda2/envs/krohitm/lib/python2.7/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [10]:
# create augmentation by feature importances as additional features
t = train['y']
tr = train.drop(['y'], axis=1)

# Tree-based estimators can be used to compute feature importances
clf = GradientBoostingRegressor(
                max_depth=4, 
                learning_rate=0.005, 
                random_state=seed, 
                subsample=0.95, 
                n_estimators=200
)

# fit regressor
clf.fit(tr, t)

# df to hold feature importances
features = pd.DataFrame()
features['feature'] = tr.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

print features.tail()

# select best features
model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(tr)

test_reduced = model.transform(test.copy())

# dataset augmentation
train = pd.concat([train, pd.DataFrame(train_reduced)], axis=1)
test = pd.concat([test, pd.DataFrame(test_reduced)], axis=1)

# check new shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))

         importance
feature            
X119       0.041776
X118       0.044236
pca_6      0.091539
X315       0.128361
X314       0.610175

Train shape: (4209, 617)
Test shape: (4209, 616)


In [11]:
# define custom R2 metrics for Keras backend
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [12]:
from keras.optimizers import SGD, Adagrad, Adadelta
# base model architecture definition
def model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  #optimizer='adam',
                  optimizer=Adadelta(),#SGD(lr=0.0001, momentum=0.9),
                  #optimizer=Adagrad(),
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [13]:
# initialize input dimension
input_dims = train.shape[1]-1

#activation functions for hidden layers
act_func = 'tanh' # could be 'relu', 'sigmoid', ...

# make np.seed fixed
np.random.seed(seed)

# initialize estimator, wrap model in KerasRegressor
estimator = KerasRegressor(
    build_fn=model, 
    nb_epoch=100, 
    batch_size=10,
    verbose=1
)

In [14]:
# X, y preparation
X, y = train.drop('y', axis=1).values, train.y.values
print(X.shape)

# X_test preparation
X_test = test
print(X_test.shape)

# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X, 
    y, 
    test_size=0.20, 
    random_state=seed
)

(4209, 616)
(4209, 616)


In [15]:
# define path to save model
import os
model_path = 'keras_model_adadelta.h5'

In [16]:
# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=10, # was 10
        verbose=1),
    
    ModelCheckpoint(
        model_path, 
        monitor='val_loss', 
        save_best_only=True, 
        verbose=0)
]

# fit estimator
#estimator.fit(
#    X_tr, 
#    y_tr, 
    #nb_epoch=10, # increase it to 20-100 to get better results
#    validation_data=(X_val, y_val),
#    verbose=2,
#    callbacks=callbacks,
#    shuffle=True
#)

In [17]:
#reload the saved model
estimator = model()
estimator.compile(loss='mean_squared_error', 
              optimizer=Adadelta(),
              metrics=[r2_keras] 
             )
estimator.load_weights('keras_model_adadelta_0627.h5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 616)           380072      dense_input_1[0][0]              
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 616)           2464        dense_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 616)           0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 616)           0           activation_1[0][0]               
___________________________________________________________________________________________

In [18]:
# check the results of saved model on train data
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

MSE train: 7.80798918632
R^2 train: 0.623668164219
MSE val: 7.92366340175
R^2 val: 0.596630478318


In [22]:
#get predictions for training data using the saved model
y_tr_predict_keras = estimator.predict(X)
#print y_tr_predict_keras.shape
y_test_keras = estimator.predict(X_test.values)

### XGBoost

In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb

In [24]:
# read datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [25]:
from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_train = train["y"]
y_mean = np.mean(y_train)

In [26]:
X, y = train.drop('y', axis=1).values, train.y.values
print(X.shape)

(4209, 397)


In [27]:
from sklearn.externals import joblib
xgb_model = joblib.load("xgb_650_GCV0625.joblib.dat")

In [28]:
y_tr_pred_xgb = xgb_model.predict(X)
print y_tr_pred_xgb.shape

(4209,)


In [29]:
# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y, y_tr_pred_xgb)**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y, y_tr_pred_xgb))) # R^2 train

MSE train: 7.69193315385
R^2 train: 0.63188961146


In [30]:
x_test = np.array(test)
y_test_xgb = xgb_model.predict(x_test).ravel()

In [31]:
print y_tr_predict_keras.shape
print y_tr_pred_xgb.shape
print y_test_keras.shape
print y_test_xgb.shape

(4209, 1)
(4209,)
(4209, 1)
(4209,)


In [32]:
stacked_columns_train = np.column_stack((y_tr_predict_keras, y_tr_pred_xgb))
stacked_columns_test = np.column_stack((y_test_keras, y_test_xgb))
avg_train = np.mean(stacked_columns_train, axis = 1)
avg_test = np.mean(stacked_columns_test, axis = 1)

In [33]:
# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y, avg_train)**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y, avg_train))) # R^2 train

MSE train: 7.69528071934
R^2 train: 0.631569134993


In [34]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': avg_test})
#output.to_csv('../results/init_avg_results_0628.csv', index=False)

In [35]:
#from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [36]:
stacked_columns_train = np.column_stack((y_tr_predict_keras, y_tr_pred_xgb))
stacked_columns_test = np.column_stack((y_test_keras, y_test_xgb))

In [66]:
lasso_reg = Lasso(max_iter=6000)

lasso_params = {
    'max_iter': [5000, 6000, 7000],
    'alpha': [1.55, 1.57, 1.6],
    'fit_intercept': [True,False],
    'normalize': [True, False],
    'precompute': [True, False],
    'tol': [0.004, 0.0045, 0.005],
    'selection': ['random', 'cyclic']
}

lasso_reg_cv = GridSearchCV(lasso_reg, lasso_params, cv = 5, verbose=1, n_jobs = 5)
lasso_reg_cv.fit(stacked_columns_train, y)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=5)]: Done 758 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done 2160 out of 2160 | elapsed:    3.3s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=6000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'normalize': [True, False], 'selection': ['random', 'cyclic'], 'fit_intercept': [True, False], 'max_iter': [5000, 6000, 7000], 'precompute': [True, False], 'tol': [0.004, 0.0045, 0.005], 'alpha': [1.55, 1.57, 1.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [67]:
print(lasso_reg_cv.best_score_)
print(lasso_reg_cv.best_params_)

means = lasso_reg_cv.cv_results_['mean_test_score']
stds = lasso_reg_cv.cv_results_['std_test_score']
params = lasso_reg_cv.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.644424282788
{'normalize': False, 'selection': 'random', 'fit_intercept': True, 'max_iter': 6000, 'precompute': False, 'tol': 0.004, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, 'selection': 'random', 'fit_intercept': True, 'max_iter': 5000, 'precompute': True, 'tol': 0.004, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, 'selection': 'random', 'fit_intercept': True, 'max_iter': 5000, 'precompute': True, 'tol': 0.0045, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, 'selection': 'random', 'fit_intercept': True, 'max_iter': 5000, 'precompute': True, 'tol': 0.005, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, 'selection': 'cyclic', 'fit_intercept': True, 'max_iter': 5000, 'precompute': True, 'tol': 0.004, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, 'selection': 'cyclic', 'fit_intercept': True, 'max_iter': 5000, 'precompute': True, 'tol': 0.0045, 'alpha': 1.55}
-0.008255 (0.011708) with: {'normalize': True, '

In [37]:
lasso_reg = Lasso(max_iter=6000, normalize= False, selection='random', 
                  fit_intercept=True, precompute=True, 
                  tol=0.004, alpha= 1.57)
lasso_reg.fit(stacked_columns_train, y)

Lasso(alpha=1.57, copy_X=True, fit_intercept=True, max_iter=6000,
   normalize=False, positive=False, precompute=True, random_state=None,
   selection='random', tol=0.004, warm_start=False)

In [38]:
y_lasso = lasso_reg.predict(stacked_columns_train)

In [39]:
# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y, y_lasso)**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y, y_lasso))) # R^2 train

MSE train: 7.65630243812
R^2 train: 0.635292048318


In [40]:
y_test_lasso = lasso_reg.predict(stacked_columns_test)

In [72]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_test_lasso})
#output.to_csv('../results/lasso_stack_0628.csv', index=False)

In [81]:
avg_results = np.mean(np.column_stack((stacked_columns_test, y_test_lasso)), axis = 1)

In [82]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': avg_results})
#output.to_csv('../results/avg_results_0628.csv', index=False)

## Trying stacking with Ridge regression

In [44]:
from sklearn.linear_model import Ridge

Ridge_reg = Ridge(random_state=seed)

Ridge_params = {
    'alpha': [0.1, 1.0, 10.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'max_iter': [500, 1000, 2000],
    'tol': [0.001, 0.01, 0.05, 0.1]
}

Ridge_cv = GridSearchCV(Ridge_reg, Ridge_params, cv = 5, verbose=1, n_jobs = -1)
Ridge_cv.fit(stacked_columns_train, y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:    1.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'normalize': [True, False], 'alpha': [0.1, 1.0, 10.0], 'max_iter': [500, 1000, 2000], 'tol': [0.001, 0.01, 0.05, 0.1], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [45]:
print(Ridge_cv.best_score_)
print(Ridge_cv.best_params_)

means = Ridge_cv.cv_results_['mean_test_score']
stds = Ridge_cv.cv_results_['std_test_score']
params = Ridge_cv.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.641113970393
{'normalize': False, 'alpha': 10.0, 'max_iter': 500, 'tol': 0.001, 'fit_intercept': True}
0.639143 (0.059285) with: {'normalize': True, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.001, 'fit_intercept': True}
0.639143 (0.059285) with: {'normalize': True, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.01, 'fit_intercept': True}
0.639143 (0.059285) with: {'normalize': True, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.05, 'fit_intercept': True}
0.639143 (0.059285) with: {'normalize': True, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.1, 'fit_intercept': True}
0.641109 (0.057205) with: {'normalize': False, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.001, 'fit_intercept': True}
0.641109 (0.057205) with: {'normalize': False, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.01, 'fit_intercept': True}
0.641109 (0.057205) with: {'normalize': False, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.05, 'fit_intercept': True}
0.641109 (0.057205) with: {'normalize': False, 'alpha': 0.1, 'max_iter': 500, 'tol': 0.1, 'fit_inter

In [ ]:
Ridge_params = {
    'alpha': [7.5, 10.0, 12.5],
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'max_iter': [500, 1000, 2000],
    'tol': [0.001, 0.01, 0.05, 0.1]
}

Ridge_cv = GridSearchCV(Ridge_reg, Ridge_params, cv = 5, verbose=1, n_jobs = -1)
Ridge_cv.fit(stacked_columns_train, y)

In [ ]:
print(Ridge_cv.best_score_)
print(Ridge_cv.best_params_)

means = Ridge_cv.cv_results_['mean_test_score']
stds = Ridge_cv.cv_results_['std_test_score']
params = Ridge_cv.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Trying stacking with SVR

In [71]:
avg_all_results = np.sum(np.column_stack((3*stacked_columns_test, 2*y_test_lasso, y_test_rbf)), axis = 1)/8.0

In [68]:
print avg_all_results

[  79.92770386   97.54924011   78.85188293 ...,   92.44129944  111.06360626
   92.24983978]


In [72]:
print avg_all_results

[  91.00233882  115.13547242   88.58579913 ...,  103.70529185  124.6283283
  103.40571257]


In [70]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': avg_all_results})
output.to_csv('../results/avg_strng_0628.csv', index=False)

In [73]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_test_rbf})
output.to_csv('../results/svr_rbf.csv', index=False)